In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

data = {
    'User': ['User1', 'User2', 'User3', 'User4', 'user5'],
    'Item1': [5, 4, np.nan, 7, 1],
    'Item2': [6, np.nan, 3, 4, np.nan],
    'Item3': [7, 3, 4, 3, 3],
    'Item4': [4, np.nan, 1, 6, 2],
    'Item5': [3, 5, 1, np.nan, 2],
    'Item6': [np.nan, 4, np.nan, 4, 5],
}

df = pd.DataFrame(data).set_index('User')
#user_similarity = df.T.corr(method='pearson')  # Pearson correlation
#print(user_similarity)


# Mean-center the ratings
mean_ratings = df.mean(axis=1)  # Calculate the mean rating for each user
mean_centered_df = df.sub(mean_ratings, axis=0)  # Subtract mean ratings from the original ratings

# Compute user similarity using mean-centered ratings
user_similarity = mean_centered_df.T.corr(method='pearson')
print("user_similarity mean",user_similarity)
user_similarity_df = pd.DataFrame(user_similarity, index=df.index, columns=df.index)

def predict_rating(user, item):
    # Filter out NaN values from the user similarity and item ratings
    similarity_scores = user_similarity_df[user]

    item_ratings = mean_centered_df[item]


    # Only consider non-NaN ratings
    mask = ~item_ratings.isna()& (similarity_scores >= 0)
    similarity_scores = similarity_scores[mask]

    item_ratings = item_ratings[mask]


    if similarity_scores.empty or item_ratings.empty:
        return 0  # Return 0 if no relevant data for prediction

    numerator = np.sum(similarity_scores * item_ratings)

    denominator = np.sum(np.abs(similarity_scores))


    if denominator == 0:
        return 0
    return mean_ratings[user]+numerator / denominator

def recommend_items(user):
    unrated_items = df.columns[pd.isna(df.loc[user])]  # Items that the user hasn't rated
    predictions = [predict_rating(user, item) for item in unrated_items]
    recommendations = pd.DataFrame({'Item': unrated_items, 'Prediction': predictions})
    return recommendations.sort_values(by='Prediction', ascending=False)

user_to_recommend = 'User2'
recommendations = recommend_items(user_to_recommend)
print(f"Recommendations for {user_to_recommend}:")
print(recommendations)

user_similarity mean User      User1     User2     User3     User4     user5
User                                                   
User1  1.000000 -1.000000  0.973729 -0.848528  0.478091
User2 -1.000000  1.000000 -1.000000  0.693375 -0.239046
User3  0.973729 -1.000000  1.000000 -1.000000  1.000000
User4 -0.848528  0.693375 -1.000000  1.000000 -0.748331
user5  0.478091 -0.239046  1.000000 -0.748331  1.000000
Recommendations for User2:
    Item  Prediction
1  Item4         5.2
0  Item2         3.2
